In [1]:
from transformer import Transformer
import torch
import numpy as np

In [75]:
help(Transformer)

Help on class Transformer in module transformer:

class Transformer(torch.nn.modules.module.Module)
 |  Transformer(d_model, ffn_hidden, num_heads, drop_prob, num_layers, max_sequence_length, target_vocab_size, input_to_index, target_to_index, START_TOKEN, END_TOKEN, PADDING_TOKEN)
 |  
 |  Method resolution order:
 |      Transformer
 |      torch.nn.modules.module.Module
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, d_model, ffn_hidden, num_heads, drop_prob, num_layers, max_sequence_length, target_vocab_size, input_to_index, target_to_index, START_TOKEN, END_TOKEN, PADDING_TOKEN)
 |      Initialize internal Module state, shared by both nn.Module and ScriptModule.
 |  
 |  forward(self, x, y, encoder_self_attention_mask=None, decoder_self_attention_mask=None, decoder_cross_attention_mask=None, enc_start_token=False, enc_end_token=False, dec_start_token=False, dec_end_token=False)
 |      Define the computation performed at every call.
 |      
 |      

In [2]:
input_file = 'input_sentences.txt'
target_file = 'target_sentences.txt'

In [3]:
START_TOKEN = '<START>'
PADDING_TOKEN = '<PADDING>'
END_TOKEN = '<END>'

In [4]:
with open('input_sentences.txt', 'r', encoding='utf-8') as f:
    text=f.read()

In [61]:
len(text)

21426

In [5]:
input_vocab = sorted(list(set(text)))
input_vocab.remove('\n')


In [8]:
input_vocab.insert(0,START_TOKEN)
input_vocab.insert(len(input_vocab),PADDING_TOKEN)
input_vocab.insert(len(input_vocab),END_TOKEN)

In [7]:
input_vocab = [char for char in input_vocab if not char.isupper()]
    

In [6]:
input_vocab.remove('"')

In [9]:
print(input_vocab)

['<START>', ' ', '!', "'", ',', '-', '.', '0', '1', '2', '3', '4', '5', '7', '8', '9', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '<PADDING>', '<END>']


In [10]:
with open('target_sentences.txt', 'r', encoding= 'utf-8') as f:
    text2=f.read()

In [11]:
target_vocab = sorted(list(set(text2)))
target_vocab.remove('\n')


In [17]:
target_vocab.insert(0,START_TOKEN)
target_vocab.insert(len(target_vocab),PADDING_TOKEN)
target_vocab.insert(len(target_vocab),END_TOKEN)

In [13]:
target_vocab = [char for char in target_vocab if not char.islower()]

In [15]:
target_vocab = [char.lower() if len(char) == 1 and char.isupper() else char for char in target_vocab]

In [18]:
print(target_vocab)

['<START>', ' ', '!', "'", '.', '0', '1', '2', '3', '4', '5', '7', '8', '9', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '<PADDING>', '<END>']


In [19]:
with open(input_file, 'r') as file:
    input_sentences = file.readlines()
with open(target_file, 'r') as file:
    target_sentences = file.readlines()

input_sentences = [sentence.rstrip('\n') for sentence in input_sentences]
target_sentences = [sentence.rstrip('\n') for sentence in target_sentences]

In [20]:
input_sentences = [sentence.lower() for sentence in input_sentences]
target_sentences = [sentence.lower() for sentence in target_sentences]

In [19]:
input_sentences = [sentence.replace('"','') for sentence in input_sentences]
target_sentences = [sentence.replace('"','') for sentence in target_sentences]

In [20]:
print(len(input_sentences))

594


In [44]:
print(input_sentences_lowered[200])


when i was a girl scout, i learned how to build a fire.


In [18]:
trial = input_sentences[0]
t=trial.lower()
print(t)
print(trial.lower())
print(trial)

my sister is having another baby.
my sister is having another baby.
My sister is having another baby.


In [21]:
print(len(target_sentences))

594


In [22]:
max(len(x) for x in input_sentences), max(len(x) for x in target_sentences)

(76, 67)

In [23]:
max_sequence_length = 77

<h1>Load sentences to Torch Dataset</h1>

In [24]:
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):

    def __init__(self, input_sentences, target_sentences):
        self.input_sentences = input_sentences
        self.target_sentences = target_sentences

    def __len__(self):
        return len(self.input_sentences)

    def __getitem__(self, idx):
        return self.input_sentences[idx], self.target_sentences[idx]

In [25]:
dataset = TextDataset(input_sentences, target_sentences)

In [26]:
len(dataset)

594

In [93]:
dataset[:10]

(['My sister is having another baby.',
  'My daughter gave me beautiful flowers ',
  'My brother is thin because he exercises everyday.',
  "I enjoy playing with my cousins at my grandmother's house.",
  'My sister is Deaf and goes to Gallaudet University.',
  'Luckily my family is very supportive.',
  'My grandfather is a dairy farmer.',
  'My father-in-law worked at a factory for twenty-five years.',
  'My family grows watermelon in our garden.',
  'Every Tuesday my family has game night.'],
 ['ANOTHER BABY MY SISTER BORN WILL.',
  'BEAUTIFUL FLOWERS MY DAUGHTER GIVE ME.',
  'MY BROTHER THIN WHY? EXERCISE EVERYDAY.',
  "GRANDMOTHER'S HOUSE COUSINS PLAY AROUND I ENJOY.",
  'GALLAUDET UNIVERSITY MY DEAF SISTER GO TO.',
  'LUCKY ME WHY? MY FAMILY SUPPORT ME!',
  'MY GRANDFATHER WORK WHAT? DAIRY FARMER.',
  '25 YEARS FACTORY MY FATHER IN LAW WORK WORK WORK.',
  'OUR G A R D E N WATERMELON MY FAMILY GROW GROW GROW.',
  'EVERY TUESDAY MY FAMILY DO DO? GAME NIGHT.'])

# Making batches

In [27]:
batch_size = 3
train_loader = DataLoader(dataset, batch_size)
iterator = iter(train_loader)

In [28]:
len(train_loader)

198

In [29]:
for batch_num,batch in enumerate(iterator):
    print(batch)
    if(batch_num > 3):
        break

[('my sister is having another baby.', 'my daughter gave me beautiful flowers ', 'my brother is thin because he exercises everyday.'), ('another baby my sister born will.', 'beautiful flowers my daughter give me.', 'my brother thin why? exercise everyday.')]
[("i enjoy playing with my cousins at my grandmother's house.", 'luckily my family is very supportive.', 'my grandfather is a dairy farmer.'), ("grandmother's house cousins play around i enjoy.", 'lucky me why? my family support me!', 'my grandfather work what? dairy farmer.')]
[('my father in law worked at a factory for twenty five years.', 'my family grows watermelon in our garden.', 'every tuesday my family has game night.'), ('25 years factory my father in law work.', 'our garden watermelon my family grow.', 'every tuesday my family do do? game night.')]
[('my daughter is always happy.', 'my mother-in-law has been very sick recently.', 'my grandfather is a silly man.'), ('my daughter she always happy.', 'my mother in law recent

# Tokenization

In [37]:
index_to_target = {k:v for k,v in enumerate(target_vocab)}
target_to_index = {v:k for k,v in enumerate(target_vocab)}
index_to_input = {k:v for k,v in enumerate(input_vocab)}
input_to_index = {v:k for k,v in enumerate(input_vocab)}

In [ ]:
target_to_index[

In [31]:
def tokenize(sentence, language_to_index, start_token=True, end_token=True):
    sentence_word_indicies = [language_to_index[token] for token in list(sentence)]
    if start_token:
        sentence_word_indicies.insert(0, language_to_index[START_TOKEN])
    if end_token:
        sentence_word_indicies.append(language_to_index[END_TOKEN])
    for _ in range(len(sentence_word_indicies), max_sequence_length):
        sentence_word_indicies.append(language_to_index[PADDING_TOKEN])
    return torch.tensor(sentence_word_indicies)

In [38]:
input_tokenized, target_tokenized = [], []
for sentence_num in range(batch_size):
    input_sentence, target_sentence = batch[0][sentence_num], batch[1][sentence_num]
    input_tokenized.append( tokenize(input_sentence, input_to_index, start_token=False, end_token=False) )
    target_tokenized.append( tokenize(target_sentence, target_to_index, start_token=True, end_token=True) )
input_tokenized = torch.stack(input_tokenized)
target_tokenized = torch.stack(target_tokenized)

In [39]:
target_tokenized

tensor([[ 0, 16, 19, 33, 34,  1, 17, 15, 25, 19, 33,  1, 27, 15, 25, 19,  1, 37,
         22, 29, 14,  1, 27, 39,  1, 15, 35, 28, 34,  4, 42, 41, 41, 41, 41, 41,
         41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41,
         41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41,
         41, 41, 41, 41, 41],
        [ 0, 26, 15, 37,  1, 33, 17, 22, 29, 29, 26,  1, 27, 39,  1, 33, 29, 28,
          1, 21, 29,  4, 42, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41,
         41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41,
         41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41,
         41, 41, 41, 41, 41],
        [ 0, 19, 36, 19, 32, 39,  1, 39, 19, 15, 32,  1, 27, 39,  1, 16, 23, 32,
         34, 22, 18, 15, 39,  1, 15, 30, 30, 26, 19,  1, 30, 23, 19,  1, 27, 39,
          1, 21, 32, 15, 28, 18, 27, 15,  1, 16, 15, 25, 19,  4, 42, 41, 41, 41,
         41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 

# Define Transformer Model

In [52]:
d_model = 512
batch_size = 20
ffn_hidden = 1024
num_heads = 8
drop_prob = 0.1
num_layers = 2
max_sequence_length = 76
target_vocab_size = len(target_vocab)

transformer = Transformer(d_model, 
                          ffn_hidden,
                          num_heads, 
                          drop_prob, 
                          num_layers, 
                          max_sequence_length,
                          target_vocab_size,
                          input_to_index,
                          target_to_index,
                          START_TOKEN, 
                          END_TOKEN, 
                          PADDING_TOKEN)

In [53]:
from torch import nn

criterian = nn.CrossEntropyLoss(ignore_index=target_to_index[PADDING_TOKEN],
                                reduction='none')

# When computing the loss, we are ignoring cases when the label is the padding token
for params in transformer.parameters():
    if params.dim() > 1:
        nn.init.xavier_uniform_(params)

optim = torch.optim.Adam(transformer.parameters(), lr=1e-4)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [42]:
transformer

Transformer(
  (encoder): Encoder(
    (sentence_embedding): SentenceEmbedding(
      (embedding): Embedding(45, 256)
      (position_encoder): PositionalEncoding()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (layers): SequentialEncoder(
      (0): EncoderLayer(
        (attention): MultiHeadAttention(
          (qkv_layer): Linear(in_features=256, out_features=768, bias=True)
          (linear_layer): Linear(in_features=256, out_features=256, bias=True)
        )
        (norm1): LayerNormalization()
        (dropout1): Dropout(p=0.1, inplace=False)
        (ffn): PositionwiseFeedForward(
          (linear1): Linear(in_features=256, out_features=1024, bias=True)
          (linear2): Linear(in_features=1024, out_features=256, bias=True)
          (relu): ReLU()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (norm2): LayerNormalization()
        (dropout2): Dropout(p=0.1, inplace=False)
      )
      (1): EncoderLayer(
        (attention): MultiHeadAtt

In [54]:
device

device(type='cuda')

In [44]:
NEG_INFTY = -1e9

def create_masks(inp_batch, tg_batch):
    num_sentences = len(inp_batch)
    look_ahead_mask = torch.full([max_sequence_length, max_sequence_length] , True)
    look_ahead_mask = torch.triu(look_ahead_mask, diagonal=1)
    encoder_padding_mask = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_self_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_cross_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)

    for idx in range(num_sentences):
      inp_sentence_length, tg_sentence_length = len(inp_batch[idx]), len(tg_batch[idx])
      inp_chars_to_padding_mask = np.arange(inp_sentence_length + 1, max_sequence_length)
      tg_chars_to_padding_mask = np.arange(tg_sentence_length + 1, max_sequence_length)
      encoder_padding_mask[idx, :, inp_chars_to_padding_mask] = True
      encoder_padding_mask[idx, inp_chars_to_padding_mask, :] = True
      decoder_padding_mask_self_attention[idx, :, tg_chars_to_padding_mask] = True
      decoder_padding_mask_self_attention[idx, tg_chars_to_padding_mask, :] = True
      decoder_padding_mask_cross_attention[idx, :, inp_chars_to_padding_mask] = True
      decoder_padding_mask_cross_attention[idx, tg_chars_to_padding_mask, :] = True

    encoder_self_attention_mask = torch.where(encoder_padding_mask, NEG_INFTY, 0)
    decoder_self_attention_mask =  torch.where(look_ahead_mask + decoder_padding_mask_self_attention, NEG_INFTY, 0)
    decoder_cross_attention_mask = torch.where(decoder_padding_mask_cross_attention, NEG_INFTY, 0)
    return encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask
     

In [55]:
transformer.train()
transformer.to(device)
total_loss = 0
num_epochs = 100

for epoch in range(num_epochs):
    print(f"Epoch {epoch}")
    iterator = iter(train_loader)
    for batch_num, batch in enumerate(iterator):
        transformer.train()
        inp_batch, tg_batch = batch
        encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask = create_masks(inp_batch, tg_batch)
        optim.zero_grad()
        tg_predictions = transformer(inp_batch,
                                     tg_batch,
                                     encoder_self_attention_mask.to(device), 
                                     decoder_self_attention_mask.to(device), 
                                     decoder_cross_attention_mask.to(device),
                                     enc_start_token=False,
                                     enc_end_token=False,
                                     dec_start_token=True,
                                     dec_end_token=True)
        labels = transformer.decoder.sentence_embedding.batch_tokenize(tg_batch, start_token=False, end_token=True)
        loss = criterian(
            tg_predictions.view(-1, target_vocab_size).to(device),
            labels.view(-1).to(device)
        ).to(device)
        valid_indicies = torch.where(labels.view(-1) == target_to_index[PADDING_TOKEN], False, True)
        loss = loss.sum() / valid_indicies.sum()
        loss.backward()
        optim.step()
        #train_losses.append(loss.item())
        if batch_num % 50 == 0:
            print(f"Iteration {batch_num} : {loss.item()}")
            print(f"English: {inp_batch[0]}")
            print(f"Translated Text: {tg_batch[0]}")
            tg_sentence_predicted = torch.argmax(tg_predictions[0], axis=1)
            predicted_sentence = ""
            for idx in tg_sentence_predicted:
              if idx == target_to_index[END_TOKEN]:
                break
              predicted_sentence += index_to_target[idx.item()]
            print(f"ASL Prediction: {predicted_sentence}")


            transformer.eval()
            tg_sentence = ("",)
            inp_sentence = ("i like to play golf.",)
            for word_counter in range(max_sequence_length):
                encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_masks(inp_sentence, tg_sentence)
                predictions = transformer(inp_sentence,
                                          tg_sentence,
                                          encoder_self_attention_mask.to(device), 
                                          decoder_self_attention_mask.to(device), 
                                          decoder_cross_attention_mask.to(device),
                                          enc_start_token=False,
                                          enc_end_token=False,
                                          dec_start_token=True,
                                          dec_end_token=False)
                next_token_prob_distribution = predictions[0][word_counter] # not actual probs
                next_token_index = torch.argmax(next_token_prob_distribution).item()
                next_token = index_to_target[next_token_index]
                tg_sentence = (tg_sentence[0] + next_token, )
                if next_token == END_TOKEN:
                  break
            
            print(f"Evaluation translation (i like to play golf.) : {tg_sentence}")
            print("-------------------------------------------")

Epoch 0
Iteration 0 : 5.140735149383545
English: my sister is having another baby.
Translated Text: another baby my sister born will.
ASL Prediction: kkkkkkkfffffhfkkkfkkaahhhh99khkfhfkhxhxfkkkkkkkhkaxh!k9kxhhhaaahhhfhkfhkfkak
Evaluation translation (i like to play golf.) : ('eee         eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee      eeeeeeee',)
-------------------------------------------
Iteration 50 : 3.144522190093994
English: would you like a straw in your drink?
Translated Text: your drink straw you want?
ASL Prediction: eeee a  e  a   a  ee ee  eee    a eyeeea.e e   esee eeee eeeeeoe  a 
Evaluation translation (i like to play golf.) : ('ee                                                                          ',)
-------------------------------------------
Iteration 100 : 3.4373881816864014
English: i like books.
Translated Text: i like books.
ASL Prediction: oaaoeos    ea eeaeeaeseeear osereraearnooaaaanse trerara e eeeeo raae reeran
Evaluation translation (i like to 

In [56]:
transformer.eval()
def translate(input_sentence):
  input_sentence = (input_sentence,)
  target_sentence = ("",)
  for word_counter in range(max_sequence_length):
    encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_masks(input_sentence, target_sentence)
    predictions = transformer(input_sentence,
                              target_sentence,
                              encoder_self_attention_mask.to(device), 
                              decoder_self_attention_mask.to(device), 
                              decoder_cross_attention_mask.to(device),
                              enc_start_token=False,
                              enc_end_token=False,
                              dec_start_token=True,
                              dec_end_token=False)
    next_token_prob_distribution = predictions[0][word_counter]
    next_token_index = torch.argmax(next_token_prob_distribution).item()
    next_token = index_to_target[next_token_index]
    target_sentence = (target_sentence[0] + next_token, )
    if next_token == END_TOKEN:
      break
  return target_sentence[0]

In [57]:
translation = translate("my sister is having baby")
print(translation)

you dring wher hat? yef<END>
